# Поисковая система отзывов на лекарства (3-й интент)

В этом ноутбуке создадим поисковую систему для поиска нескольких ближайших релевантных отзывов на лекарства по заданному запросу

In [1]:
!pip install -q transformers
! pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import annoy
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import dill

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
path =  "/content/drive/My Drive/GeekBrains/NLP/Course Project/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Загрузим датасет с отзывами:

In [4]:
data = pd.read_csv(path + 'drugs_feedback.csv')
data

,text,class
0,"нам прописали, так мой ребенок сыпью покрылся,...",2
1,Общее впечатление : не подошел,2
2,Пила этот препарат для повышения иммунитета 5 ...,2
3,"Так как начала работать в аптеке, начала часто...",2
4,В месяц по нескольку раз причем со всеми вытек...,2
...,...,...
4804,"Покраснение спадает, неприятные ощущения прохо...",2
4805,"Из своего опыта отметил бы, что средство себя ...",2
4806,Из своего опыта хотел бы порекомендовать начат...,2
4807,Тогда оно может спасти от длительной болезни.,2


Посмотрим, какие максимальные длины отзывов есть:

In [5]:
data.text.apply(lambda x: len(str(x))).sort_values(ascending=False)[:10]

776     631
1980    592
4311    573
4218    531
3935    523
4548    500
3757    499
2615    479
3913    430
2443    403
Name: text, dtype: int64

Будем использовать максимальную длину токенизированного предложения 512

In [6]:
data = data.text.tolist()

In [7]:
len_data = len(data)

Загрузим токенайзер и BERT

In [8]:
bert = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased")
tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Токенизируем датасет. 2 набора используется, чтобы не переполнить ОЗУ

In [9]:
encodings1 = tokenizer(data[:len_data//2], 
                      max_length=512, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')
encodings2 = tokenizer(data[len_data//2:], 
                      max_length=512, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')

Создадим датасеты:

In [10]:
dataset1 = tf.data.Dataset.from_tensor_slices(dict(encodings1)).batch(4)
dataset2 = tf.data.Dataset.from_tensor_slices(dict(encodings2)).batch(4)

Получим ембеддинги отзывов:

In [11]:
out1 = bert.predict(dataset1, batch_size=4).pooler_output
out2 = bert.predict(dataset2, batch_size=4).pooler_output

602/602 [==============================] - 101s 167ms/step


Соединим 2 эмбеддинга:

In [20]:
out = np.concatenate([out1, out2])

Сохраним полученные эмбеддинги на случай долгого построения индекса:

In [21]:
with open (path + 'vec_drags_feedback.dill', 'wb') as file:
    dill.dump(out, file)

In [22]:
with open(path + 'vec_drags_feedback.dill', 'rb') as f:
    vectors = dill.load(f)

Получим поисковый индекс:

In [23]:
def get_annoy_index(embeddings):
    index = annoy.AnnoyIndex(768 ,'angular')
    
    counter = 0

    for vector in tqdm_notebook(embeddings):
        index.add_item(counter, vector)
        counter += 1
        
    index.build(15, n_jobs=-1)

    return index

In [24]:
index = get_annoy_index(vectors)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4809 [00:00<?, ?it/s]

Сохраним поисковый индекс для дальнейшего использования:

In [25]:
index.save(path + 'index.ann')

True

In [26]:
index = annoy.AnnoyIndex(768 ,'angular')
index.load(path + 'index.ann')

True

Проверим работу поисковой системы:

In [27]:
tokens = tokenizer('Арбидол не помогает улучшить иммунитет', 
                      max_length=512, 
                      truncation=True, 
                      padding='max_length', 
                      return_token_type_ids=False, 
                      return_tensors='tf')

In [28]:
vector = bert(**tokens).pooler_output[0]

In [29]:
answers = index.get_nns_by_vector(vector, 3)
answers

[4316, 3598, 2976]

In [30]:
data[4316]

'Иммунофлазид один из немногих препаратов, который облегчает протекание ОРВИ.'

In [31]:
data[3598]

'Цена у крема невысокая, по сравнению с другими препаратами.'

In [32]:
data[2976]

'Очень помогает когда бывает повышенный пульс от волнения или от реакции на другие факторы, которые могут вызвать тахикардию.'